# 1. Modulos

In [1]:
import datetime as dt
from datetime import datetime, date, timedelta

import os
import sys
from pathlib import Path
import pandas as pd
import openpyxl
from openpyxl import load_workbook
import xlsxwriter
import re  # Expresiones regulares para procesamiento de texto

# Añadir la carpeta raíz del proyecto al sys.path para importar módulos personalizados
#sys.path.append(os.path.abspath("c:/Users/osmarrincon/Documents/capresoca-data-automation"))
sys.path.append(os.path.abspath(r"C:\Users\crist\Documents\Proyectos Python\capresoca-data-automation"))  # Ruta alternativa (comentada)

# Importar función y clase personalizada del proyecto
from src.file_loader import cargar_maestros_ADRES  # Función para cargar archivos maestros ADRES
from src.data_cleaning import BduaReportProcessor      # Clase para limpiar y normalizar población Maestro ADRES
from src.data_cleaning import DataCleaner # Clase para limpiar y normalizar DataFrames de Pandas

# 2. Rutas y variables

In [2]:
R_Maestro__EPSC25 = r"C:\Users\crist\OneDrive - uniminuto.edu\Capresoca\AlmostClear\Procesos BDUA\Contributivo\Maestro\2025-2\EPSC25MC0015102025.TXT"
R_Maestro__EPS025 = r"C:\Users\crist\OneDrive - uniminuto.edu\Capresoca\AlmostClear\Procesos BDUA\Subsidiados\Maestro\MS\2025-02\EPS025MS0015102025.TXT"
R_IPS = r"C:\Users\crist\OneDrive - uniminuto.edu\Capresoca\AlmostClear\Constantes\IPS_CODIGO.txt"
R_Dic_Nomenclaturas = r"C:\Users\crist\OneDrive - uniminuto.edu\Capresoca\AlmostClear\Constantes\NOMENCLATURA_DE_DIRECCIONES.xlsx"
# Crear el objeto de fecha
fecha_reporte = dt.datetime(2025, 10, 17)
fecha_archivo = "17-10-2025"

Ruta_Salida = fr"C:\Users\crist\OneDrive - 891856000_CAPRESOCA E P S\Escritorio\Yesid Rincón Z\Traslados\Procesos BDUA\2025\10_Octubre\17"

S_Excel = fr"C:\Users\crist\OneDrive - 891856000_CAPRESOCA E P S\Escritorio\Yesid Rincón Z\Traslados\Procesos BDUA\2025\10_Octubre\17\DataFrames_Activos 17102025 - copia.xlsx"
hoja = "Df_NS_Envio"

# 3. Cargue Dataframes

In [3]:
# Cargar y combinar los maestros
maestro_ADRES = cargar_maestros_ADRES(R_Maestro__EPS025, R_Maestro__EPSC25)
df_NS = pd.read_excel(S_Excel, hoja, header=0, dtype=str)
df_Dic_Nomenclaturas = pd.read_excel(R_Dic_Nomenclaturas, "NOMENCLATURA", header=0, dtype=str)
df_IPS = pd.read_csv(R_IPS, sep=None, engine='python', encoding='utf-8', header=None, dtype=str)


# 4. Limpiar datos
## 4.1. Listado censal o Sisben ADRES

In [4]:
# Duplicar la columna "MARCASISBENIV+MARCASISBENIII_2" y nombrarla "MARCASISBENIV+MARCASISBENIII"
maestro_ADRES["MARCASISBENIV+MARCASISBENIII_2"] = \
    maestro_ADRES["MARCASISBENIV+MARCASISBENIII"]

# 1. Instanciar el procesador: Se crea un objeto pasando el DataFrame.
#    La jerarquía de población ya está definida por defecto dentro de la clase.
processor = BduaReportProcessor(df=maestro_ADRES)

# 2. Ejecutar la limpieza y asignarla de vuelta.
#    El método retorna un DataFrame completamente nuevo con la columna actualizada.
maestro_ADRES = processor.prioritize_population_markers(
    col_name="MARCASISBENIV+MARCASISBENIII"
)

# ¡Listo! 'maestro_ADRES' ahora contiene los datos limpios.

In [5]:
# Crear las dos nuevas columnas basadas en la columna MARCASISBENIV+MARCASISBENIII
def extraer_grupo_poblacional(valor):
	"""Extrae el grupo poblacional según las reglas especificadas"""
	if pd.isna(valor) or valor == '' or str(valor).strip() == '':
		return "No sisben"
	elif valor == "Sisben D":
		return "34"
	elif "LC(" in str(valor):
		# Extraer número entre paréntesis de LC(número)
		match = re.search(r'LC\((\d+)\)', str(valor))
		return match.group(1) if match else "No sisben"
	elif "SIV(" in str(valor):
		return "5"
	else:
		return "No sisben"

def extraer_nivel_sisben(valor):
	"""Extrae el nivel de sisben para casos SIV"""
	if pd.notna(valor) and "SIV(" in str(valor):
		# Extraer código entre paréntesis de SIV(código)
		match = re.search(r'SIV\(([^)]+)\)', str(valor))
		return match.group(1) if match else ""
	else:
		return ""

# Aplicar las funciones para crear las nuevas columnas
maestro_ADRES['Gr_Poblacional_Actual'] = maestro_ADRES['MARCASISBENIV+MARCASISBENIII'].apply(extraer_grupo_poblacional)
maestro_ADRES['N_Sisben_Actual'] = maestro_ADRES['MARCASISBENIV+MARCASISBENIII'].apply(extraer_nivel_sisben)

# Mostrar algunas filas para verificar el resultado
print("Verificación de las nuevas columnas:")
print(maestro_ADRES[['MARCASISBENIV+MARCASISBENIII', 'Gr_Poblacional_Actual', 'N_Sisben_Actual']].head(10))

Verificación de las nuevas columnas:
  MARCASISBENIV+MARCASISBENIII Gr_Poblacional_Actual N_Sisben_Actual
0                     SIV(C10)                     5             C10
1                          NaN             No sisben                
2                          NaN             No sisben                
3                          NaN             No sisben                
4                          NaN             No sisben                
5                          NaN             No sisben                
6                          NaN             No sisben                
7                     SIV(B06)                     5             B06
8                          NaN             No sisben                
9                          NaN             No sisben                


## 4.2. Estructura general de NS

In [6]:
# 1. Ordenar por la columna NOVEDAD (de menor a mayor)
df_NS = df_NS.sort_values('NOVEDAD').reset_index(drop=True)

# 2. Función mejorada para formatear fechas con detección automática de formato
def format_date_column_with_diagnosis(df, column_name, target_format='%d/%m/%Y'):
    """Estandariza formato de fecha con diagnóstico completo y detección automática de formato"""
    print(f"\n=== PROCESANDO COLUMNA: {column_name} ===")
    
    # DIAGNÓSTICO ANTES
    original_total = len(df)
    original_nulls = df[column_name].isna().sum()
    original_not_nulls = df[column_name].notna().sum()
    
    print(f"📊 ANTES DEL PROCESAMIENTO:")
    print(f"   Total registros: {original_total}")
    print(f"   Fechas vacías/nulas: {original_nulls}")
    print(f"   Fechas no vacías: {original_not_nulls}")
    print(f"   Porcentaje de vacías: {(original_nulls/original_total)*100:.2f}%")
    
    # Mostrar ejemplos de fechas originales
    if original_not_nulls > 0:
        print(f"   Ejemplos de fechas originales: {df[column_name].dropna().head(5).tolist()}")
    
    # Guardar valores originales para debugging
    original_values = df[column_name].copy()
    
    try:
        # Detectar formato automáticamente basado en una muestra
        sample_date = str(df[column_name].dropna().iloc[0]) if original_not_nulls > 0 else ""
        
        if ":" in sample_date and len(sample_date) > 10:  
            # Formato YYYY-MM-DD HH:MM:SS (viene del Excel como string)
            print("   🔍 Formato detectado: ISO con tiempo (YYYY-MM-DD HH:MM:SS)")
            converted_dates = pd.to_datetime(df[column_name], errors='coerce')
            
        elif "-" in sample_date and sample_date.count("-") == 2:
            # Formato YYYY-MM-DD (ISO)
            print("   🔍 Formato detectado: ISO (YYYY-MM-DD)")
            converted_dates = pd.to_datetime(df[column_name], errors='coerce')
            
        elif "/" in sample_date:
            # Formato DD/MM/YYYY o MM/DD/YYYY
            print("   🔍 Formato detectado: Formato con barras (/)")
            # Intentar primero con día primero (formato europeo)
            converted_dates = pd.to_datetime(df[column_name], 
                                           errors='coerce', 
                                           dayfirst=True)
        else:
            # Formato desconocido, usar detección automática
            print("   🔍 Formato desconocido, usando detección automática")
            converted_dates = pd.to_datetime(df[column_name], errors='coerce')
        
        # Aplicar formato de salida
        df[column_name] = converted_dates.dt.strftime(target_format)
        
        # DIAGNÓSTICO DESPUÉS
        final_nulls = df[column_name].isna().sum()
        final_not_nulls = df[column_name].notna().sum()
        dates_lost = original_not_nulls - final_not_nulls
        
        print(f"📈 DESPUÉS DEL PROCESAMIENTO:")
        print(f"   Fechas vacías/nulas: {final_nulls}")
        print(f"   Fechas no vacías: {final_not_nulls}")
        print(f"   Porcentaje de vacías: {(final_nulls/original_total)*100:.2f}%")
        print(f"   Fechas perdidas en el proceso: {dates_lost}")
        
        if dates_lost > 0:
            print(f"   ⚠️  SE PERDIERON {dates_lost} FECHAS EN EL PROCESO")
            # Mostrar ejemplos de fechas perdidas usando valores originales
            lost_mask = converted_dates.isna() & original_values.notna()
            if lost_mask.any():
                print(f"   Ejemplos de fechas perdidas: {original_values[lost_mask].head().tolist()}")
        else:
            print(f"   ✅ NO SE PERDIERON FECHAS")
        
        # Mostrar ejemplos de fechas finales
        if final_not_nulls > 0:
            print(f"   Ejemplos de fechas finales: {df[column_name].dropna().head(5).tolist()}")
            
    except Exception as e:
        print(f"❌ Error al formatear la columna {column_name}: {e}")
    
    return df

# Validación inicial de fechas vacías
print("🔍 VALIDACIÓN INICIAL DE FECHAS VACÍAS:")
print(f"AFL_FECHA_NACIMIENTO vacías: {df_NS['AFL_FECHA_NACIMIENTO'].isna().sum()}")
print(f"FECHA_NOVEDAD vacías: {df_NS['FECHA_NOVEDAD'].isna().sum()}")

# Aplicar formateo de fechas con diagnóstico
df_NS = format_date_column_with_diagnosis(df_NS, 'AFL_FECHA_NACIMIENTO')
df_NS = format_date_column_with_diagnosis(df_NS, 'FECHA_NOVEDAD')

# 3. Estandarizar DPR_ID a 2 dígitos con ceros a la izquierda
df_NS['DPR_ID'] = df_NS['DPR_ID'].astype(str).str.zfill(2)

# 4. Estandarizar MNS_ID a 3 dígitos con ceros a la izquierda
df_NS['MNS_ID'] = df_NS['MNS_ID'].astype(str).str.zfill(3)

# VERIFICACIÓN FINAL COMPLETA
print("\n" + "="*60)
print("📋 VERIFICACIÓN FINAL DE ESTANDARIZACIÓN:")
print("="*60)
print(f"NOVEDAD ordenada - Primeros 5 valores: {df_NS['NOVEDAD'].head().tolist()}")
print(f"AFL_FECHA_NACIMIENTO - Muestra: {df_NS['AFL_FECHA_NACIMIENTO'].head().tolist()}")
print(f"FECHA_NOVEDAD - Muestra: {df_NS['FECHA_NOVEDAD'].head().tolist()}")
print(f"DPR_ID - Valores únicos: {sorted(df_NS['DPR_ID'].unique())}")
print(f"MNS_ID - Valores únicos: {sorted(df_NS['MNS_ID'].unique())}")

# RESUMEN FINAL DE FECHAS VACÍAS
print(f"\n🎯 RESUMEN FINAL DE FECHAS VACÍAS:")
print(f"AFL_FECHA_NACIMIENTO vacías: {df_NS['AFL_FECHA_NACIMIENTO'].isna().sum()}")
print(f"FECHA_NOVEDAD vacías: {df_NS['FECHA_NOVEDAD'].isna().sum()}")
print(f"Total de registros: {len(df_NS)}")

🔍 VALIDACIÓN INICIAL DE FECHAS VACÍAS:
AFL_FECHA_NACIMIENTO vacías: 0
FECHA_NOVEDAD vacías: 0

=== PROCESANDO COLUMNA: AFL_FECHA_NACIMIENTO ===
📊 ANTES DEL PROCESAMIENTO:
   Total registros: 25704
   Fechas vacías/nulas: 0
   Fechas no vacías: 25704
   Porcentaje de vacías: 0.00%
   Ejemplos de fechas originales: ['2025-05-07 00:00:00', '2007-08-29 00:00:00', '2007-08-15 00:00:00', '2025-10-02 00:00:00', '2025-01-27 00:00:00']
   🔍 Formato detectado: ISO con tiempo (YYYY-MM-DD HH:MM:SS)
📈 DESPUÉS DEL PROCESAMIENTO:
   Fechas vacías/nulas: 0
   Fechas no vacías: 25704
   Porcentaje de vacías: 0.00%
   Fechas perdidas en el proceso: 0
   ✅ NO SE PERDIERON FECHAS
   Ejemplos de fechas finales: ['07/05/2025', '29/08/2007', '15/08/2007', '02/10/2025', '27/01/2025']

=== PROCESANDO COLUMNA: FECHA_NOVEDAD ===
📊 ANTES DEL PROCESAMIENTO:
   Total registros: 25704
   Fechas vacías/nulas: 0
   Fechas no vacías: 25704
   Porcentaje de vacías: 0.00%
   Ejemplos de fechas originales: ['2025-06-03 00

# 4.3. Ñ

In [7]:
# Función mejorada para corregir caracteres especiales con validaciones adicionales
def corregir_caracteres_especiales(df, caracteres_problematicos=['¥', '?'], caracter_correcto='Ñ'):
    """
    Corrige caracteres especiales mal codificados en todas las columnas de texto del DataFrame
    
    Args:
        df: DataFrame a procesar
        caracteres_problematicos: Lista de caracteres que se quieren reemplazar
        caracter_correcto: Caracter por el cual se van a reemplazar
    
    Returns:
        DataFrame con caracteres corregidos
    """
    print(f"🔧 INICIANDO CORRECCIÓN DE CARACTERES ESPECIALES")
    print(f"   Caracteres a corregir: {caracteres_problematicos} → {caracter_correcto}")
    
    # Validación inicial
    if df is None or df.empty:
        print("   ⚠️ DataFrame vacío o None")
        return df
    
    # Contadores
    total_cambios = 0
    cambios_por_columna = {}
    filas_modificadas = set()  # Usar set para evitar duplicados
    
    # Obtener solo columnas de tipo object (texto)
    columnas_texto = df.select_dtypes(include=['object']).columns.tolist()
    total_filas = len(df)
    
    print(f"   DataFrame shape: {df.shape}")
    print(f"   Procesando {len(columnas_texto)} columnas de texto en {total_filas} filas...")
    print(f"   Columnas de texto: {columnas_texto[:5]}{'...' if len(columnas_texto) > 5 else ''}")
    
    # Procesar cada columna de texto
    for columna in columnas_texto:
        # Validar que la columna existe
        if columna not in df.columns:
            print(f"   ⚠️ Columna '{columna}' no encontrada, saltando...")
            continue
            
        cambios_columna = 0
        
        try:
            # Para cada caracter problemático
            for char_problema in caracteres_problematicos:
                # Obtener la serie de la columna y convertir a string
                serie_columna = df[columna].astype(str)
                
                # Identificar filas con problemas ANTES del cambio
                mask_problemas = serie_columna.str.contains(char_problema, na=False, regex=False)
                ocurrencias_antes = mask_problemas.sum()
                
                if ocurrencias_antes > 0:
                    # Agregar índices de filas modificadas al set
                    indices_modificados = df[mask_problemas].index.tolist()
                    filas_modificadas.update(indices_modificados)
                    
                    # Realizar el reemplazo
                    df[columna] = serie_columna.str.replace(char_problema, caracter_correcto, regex=False)
                    cambios_columna += ocurrencias_antes
                    
                    # Mostrar ejemplos de cambios
                    if ocurrencias_antes > 0:
                        ejemplos_antes = df.loc[indices_modificados[:3], columna].tolist()
                        print(f"     📝 {columna}: {ocurrencias_antes} ocurrencias de '{char_problema}' corregidas")
                        print(f"        Ejemplos corregidos: {ejemplos_antes}")
        
        except Exception as e:
            print(f"   ❌ Error procesando columna '{columna}': {e}")
            continue
        
        if cambios_columna > 0:
            cambios_por_columna[columna] = cambios_columna
            total_cambios += cambios_columna
    
    # Estadísticas finales
    total_filas_modificadas = len(filas_modificadas)
    porcentaje_filas_modificadas = (total_filas_modificadas / total_filas) * 100 if total_filas > 0 else 0
    
    # Resumen final
    print(f"\n📊 RESUMEN DE CORRECCIONES:")
    print(f"   Total de cambios realizados: {total_cambios}")
    print(f"   Total de filas modificadas: {total_filas_modificadas}")
    print(f"   Porcentaje de filas modificadas: {porcentaje_filas_modificadas:.2f}%")
    print(f"   Filas sin modificar: {total_filas - total_filas_modificadas}")
    
    if cambios_por_columna:
        print(f"   📈 Columnas afectadas:")
        for col, cambios in cambios_por_columna.items():
            print(f"     - {col}: {cambios} cambios")
        
        # Mostrar algunos índices de filas modificadas
        if total_filas_modificadas > 0:
            indices_muestra = sorted(list(filas_modificadas))[:10]
            print(f"   📍 Ejemplos de filas modificadas (índices): {indices_muestra}")
            if total_filas_modificadas > 10:
                print(f"      ... y {total_filas_modificadas - 10} filas más")
    else:
        print(f"   ✅ No se encontraron caracteres problemáticos")
    
    return df

# Aplicar corrección a df_NS con conteo de filas
print("="*60)
print("CORRIGIENDO CARACTERES EN df_NS")
print("="*60)
df_NS = corregir_caracteres_especiales(df_NS)

print("\n" + "="*60)
print("CORRIGIENDO CARACTERES EN maestro_ADRES")
print("="*60)
maestro_ADRES = corregir_caracteres_especiales(maestro_ADRES)

# Verificación adicional mejorada con conteo
print("\n🔍 VERIFICACIÓN ESPECÍFICA PARA PALABRAS COMUNES:")
palabras_verificar = ['NIÑO', 'NIÑA', 'AÑO', 'PEÑA', 'MONTAÑA', 'ESPAÑA']

for df_name, df in [('df_NS', df_NS), ('maestro_ADRES', maestro_ADRES)]:
    print(f"\n   📋 Verificando en {df_name} ({len(df)} filas):")
    columnas_texto = df.select_dtypes(include=['object']).columns
    
    for palabra in palabras_verificar:
        total_encontradas = 0
        filas_con_palabra = set()
        
        for col in columnas_texto:
            try:
                # CORREGIDO: agregado regex=False también en la verificación
                mask_palabra = df[col].astype(str).str.contains(palabra, case=False, na=False, regex=False)
                count = mask_palabra.sum()
                total_encontradas += count
                
                if count > 0:
                    filas_con_palabra.update(df[mask_palabra].index.tolist())
            except Exception as e:
                print(f"     ⚠️ Error verificando '{palabra}' en columna '{col}': {e}")
                continue
        
        if total_encontradas > 0:
            print(f"     ✅ '{palabra}': {total_encontradas} ocurrencias en {len(filas_con_palabra)} filas")
        else:
            print(f"     ❌ '{palabra}': No encontrada")

# Resumen global final
print(f"\n🎯 RESUMEN GLOBAL DE CORRECCIONES:")
print(f"   df_NS: {len(df_NS)} registros procesados")
print(f"   maestro_ADRES: {len(maestro_ADRES)} registros procesados")
print(f"   ✅ Proceso de corrección de caracteres completado")


CORRIGIENDO CARACTERES EN df_NS
🔧 INICIANDO CORRECCIÓN DE CARACTERES ESPECIALES
   Caracteres a corregir: ['¥', '?'] → Ñ
   DataFrame shape: (25704, 23)
   Procesando 23 columnas de texto en 25704 filas...
   Columnas de texto: ['NUM_SOLICITUD_NOVEDAD', 'ENT_ID', 'TPS_IDN_ID', 'HST_IDN_NUMERO_IDENTIFICACION', 'AFL_PRIMER_APELLIDO']...
     📝 AFL_PRIMER_APELLIDO: 120 ocurrencias de '¥' corregidas
        Ejemplos corregidos: ['CAHUEÑO', 'MONTAÑEZ', 'MONTAÑEZ']
     📝 AFL_PRIMER_APELLIDO: 2 ocurrencias de '?' corregidas
        Ejemplos corregidos: ['CAHUEÑO', 'CAHUEÑO']
     📝 AFL_SEGUNDO_APELLIDO: 116 ocurrencias de '¥' corregidas
        Ejemplos corregidos: ['CASTAÑEDA', 'CASTAÑEDA', 'RIAÑO']
     📝 AFL_SEGUNDO_APELLIDO: 4 ocurrencias de '?' corregidas
        Ejemplos corregidos: ['RIAÑO', 'CASTAÑEDA', 'CASTAÑEDA']
     📝 COD_1_NOVEDAD: 6 ocurrencias de '?' corregidas
        Ejemplos corregidos: ['MONTAÑEZ', 'NIÑO', 'ESTUPIÑAN']
     📝 COD_2_NOVEDAD: 1 ocurrencias de '?' corregidas

# 5. Procesar novedades
## 5.1. N01

In [8]:
def validar_novedades_N01(df):
    """
    Valida y corrige las columnas COD_2_NOVEDAD, COD_3_NOVEDAD, COD_4_NOVEDAD 
    para registros con NOVEDAD = 'N01'
    """
    print("🔍 INICIANDO VALIDACIÓN DE NOVEDADES N01")
    print("="*60)
    
    # Filtrar solo registros N01
    mask_n01 = df['NOVEDAD'] == 'N01'
    registros_n01 = df[mask_n01].copy()
    total_n01 = len(registros_n01)
    
    print(f"📊 Total de registros N01: {total_n01}")
    
    if total_n01 == 0:
        print("⚠️ No se encontraron registros N01")
        return df
    
    # ========== VALIDAR COD_2_NOVEDAD (debe ser entero) ==========
    print(f"\n🔧 VALIDANDO COD_2_NOVEDAD (debe ser entero)")
    
    # Contar valores vacíos antes
    vacios_cod2_antes = registros_n01['COD_2_NOVEDAD'].isna().sum()
    print(f"   Valores vacíos antes: {vacios_cod2_antes}")
    
    # Mostrar ejemplos de valores actuales
    ejemplos_cod2 = registros_n01['COD_2_NOVEDAD'].dropna().head(10).tolist()
    print(f"   Ejemplos actuales: {ejemplos_cod2}")
    
    # Validar si son enteros válidos
    def es_entero_valido(valor):
        if pd.isna(valor) or str(valor).strip() == '':
            return False
        try:
            int(str(valor).strip())
            return True
        except:
            return False
    
    # Aplicar validación
    registros_n01['COD_2_VALIDO'] = registros_n01['COD_2_NOVEDAD'].apply(es_entero_valido)
    invalidos_cod2 = (~registros_n01['COD_2_VALIDO']).sum()
    print(f"   Valores inválidos encontrados: {invalidos_cod2}")
    
    if invalidos_cod2 > 0:
        ejemplos_invalidos = registros_n01[~registros_n01['COD_2_VALIDO']]['COD_2_NOVEDAD'].head(5).tolist()
        print(f"   Ejemplos de valores inválidos: {ejemplos_invalidos}")
    
    # ========== VALIDAR COD_3_NOVEDAD (debe ser fecha DD/MM/YYYY) ==========
    print(f"\n🔧 VALIDANDO COD_3_NOVEDAD (debe ser fecha DD/MM/YYYY)")
    
    # Contar valores vacíos antes
    vacios_cod3_antes = registros_n01['COD_3_NOVEDAD'].isna().sum()
    print(f"   Valores vacíos antes: {vacios_cod3_antes}")
    
    # Mostrar ejemplos de valores actuales
    ejemplos_cod3 = registros_n01['COD_3_NOVEDAD'].dropna().head(10).tolist()
    print(f"   Ejemplos actuales: {ejemplos_cod3}")
    
    def convertir_fecha_cod3(valor):
        """Convierte diferentes formatos de fecha a DD/MM/YYYY"""
        if pd.isna(valor) or str(valor).strip() == '':
            return None
        
        valor_str = str(valor).strip()
        
        try:
            # Si es un número (formato Excel serializado)
            if valor_str.isdigit() and len(valor_str) >= 5:
                # Convertir número Excel a fecha
                fecha_excel = pd.to_datetime('1900-01-01') + pd.Timedelta(days=int(valor_str)-2)
                return fecha_excel.strftime('%d/%m/%Y')
            
            # Si ya tiene formato de fecha
            elif '/' in valor_str or '-' in valor_str:
                # Intentar convertir con diferentes formatos
                fecha_convertida = pd.to_datetime(valor_str, dayfirst=True, errors='coerce')
                if pd.notna(fecha_convertida):
                    return fecha_convertida.strftime('%d/%m/%Y')
                else:
                    return None
            else:
                return None
                
        except Exception as e:
            return None
    
    # Aplicar conversión de fechas
    registros_n01['COD_3_CONVERTIDA'] = registros_n01['COD_3_NOVEDAD'].apply(convertir_fecha_cod3)
    
    # Contar fechas válidas después de conversión
    fechas_validas = registros_n01['COD_3_CONVERTIDA'].notna().sum()
    fechas_invalidas = registros_n01['COD_3_CONVERTIDA'].isna().sum()
    
    print(f"   Fechas válidas después de conversión: {fechas_validas}")
    print(f"   Fechas que no pudieron convertirse: {fechas_invalidas}")
    
    if fechas_invalidas > 0:
        ejemplos_invalidos_fecha = registros_n01[registros_n01['COD_3_CONVERTIDA'].isna()]['COD_3_NOVEDAD'].head(5).tolist()
        print(f"   Ejemplos de fechas inválidas: {ejemplos_invalidos_fecha}")
    
    # ========== VALIDAR COD_4_NOVEDAD (no puede estar vacío) ==========
    print(f"\n🔧 VALIDANDO COD_4_NOVEDAD (no puede estar vacío)")
    
    vacios_cod4_antes = registros_n01['COD_4_NOVEDAD'].isna().sum()
    print(f"   Valores vacíos antes: {vacios_cod4_antes}")
    
    # Mostrar ejemplos de valores actuales
    ejemplos_cod4 = registros_n01['COD_4_NOVEDAD'].dropna().head(10).tolist()
    print(f"   Ejemplos actuales: {ejemplos_cod4}")
    
    # ========== APLICAR CORRECCIONES AL DATAFRAME PRINCIPAL ==========
    print(f"\n🔄 APLICANDO CORRECCIONES AL DATAFRAME PRINCIPAL")
    
    # Aplicar corrección de fechas
    df.loc[mask_n01, 'COD_3_NOVEDAD'] = registros_n01['COD_3_CONVERTIDA']
    
    # ========== RESUMEN FINAL DE VALIDACIÓN ==========
    print(f"\n📋 RESUMEN FINAL DE VALIDACIÓN N01:")
    print(f"   Total registros N01: {total_n01}")
    
    # Revalidar después de correcciones
    mask_n01_final = df['NOVEDAD'] == 'N01'
    registros_n01_final = df[mask_n01_final].copy()
    
    # COD_2_NOVEDAD final
    cod2_vacios_final = registros_n01_final['COD_2_NOVEDAD'].isna().sum()
    print(f"   COD_2_NOVEDAD vacíos: {cod2_vacios_final} ({'✅' if cod2_vacios_final == 0 else '❌'})")
    
    # COD_3_NOVEDAD final
    cod3_vacios_final = registros_n01_final['COD_3_NOVEDAD'].isna().sum()
    print(f"   COD_3_NOVEDAD vacíos: {cod3_vacios_final} ({'✅' if cod3_vacios_final == 0 else '❌'})")
    
    # COD_4_NOVEDAD final
    cod4_vacios_final = registros_n01_final['COD_4_NOVEDAD'].isna().sum()
    print(f"   COD_4_NOVEDAD vacíos: {cod4_vacios_final} ({'✅' if cod4_vacios_final == 0 else '❌'})")
    
    # Mostrar registros problemáticos si los hay
    problematicos = registros_n01_final[
        registros_n01_final['COD_2_NOVEDAD'].isna() | 
        registros_n01_final['COD_3_NOVEDAD'].isna() | 
        registros_n01_final['COD_4_NOVEDAD'].isna()
    ]
    
    if len(problematicos) > 0:
        print(f"\n⚠️ REGISTROS PROBLEMÁTICOS ENCONTRADOS: {len(problematicos)}")
        print("   Primeros 5 registros con problemas:")
        cols_mostrar = ['NUM_SOLICITUD_NOVEDAD', 'HST_IDN_NUMERO_IDENTIFICACION', 'COD_2_NOVEDAD', 'COD_3_NOVEDAD', 'COD_4_NOVEDAD']
        print(problematicos[cols_mostrar].head())
    else:
        print(f"   ✅ Todos los registros N01 tienen datos completos")
    
    return df

# Ejecutar validación
df_NS = validar_novedades_N01(df_NS)

🔍 INICIANDO VALIDACIÓN DE NOVEDADES N01
📊 Total de registros N01: 20

🔧 VALIDANDO COD_2_NOVEDAD (debe ser entero)
   Valores vacíos antes: 0
   Ejemplos actuales: ['1118583296', '1085048753', '1123436386', '1222148008', '1115921012', '1117327408', '1116559207', '1118583296', '1115921012', '63321752']
   Valores inválidos encontrados: 0

🔧 VALIDANDO COD_3_NOVEDAD (debe ser fecha DD/MM/YYYY)
   Valores vacíos antes: 0
   Ejemplos actuales: ['07/05/2025', '29/08/2007', '15/08/2007', '02/10/2025', '27/01/2025', '30/09/2025', '07/10/2025', '45784', '45684', '14/03/1965']
   Fechas válidas después de conversión: 20
   Fechas que no pudieron convertirse: 0

🔧 VALIDANDO COD_4_NOVEDAD (no puede estar vacío)
   Valores vacíos antes: 0
   Ejemplos actuales: ['1', '1', '0', '0', '1', '0', '0', '1', '1', '1']

🔄 APLICANDO CORRECCIONES AL DATAFRAME PRINCIPAL

📋 RESUMEN FINAL DE VALIDACIÓN N01:
   Total registros N01: 20
   COD_2_NOVEDAD vacíos: 0 (✅)
   COD_3_NOVEDAD vacíos: 0 (✅)
   COD_4_NOVEDAD v

# GUARDAR 

In [9]:
print("Valores únicos en la columna NOVEDAD:")
print(df_NS['NOVEDAD'].unique())

Valores únicos en la columna NOVEDAD:
['N01' 'N02' 'N03' 'N04' 'N09' 'N12' 'N14' 'N19' 'N21' 'N25' 'N31' 'N32'
 'N36' 'N39' 'N41' 'N43' 'N46']


In [10]:
maestro_ADRES

,AFL_ID,ENT_ID,TPS_IDN_ID_CF,HST_IDN_NUMERO_IDENTIFICACION_CF,TPS_IDN_ID,HST_IDN_NUMERO_IDENTIFICACION,AFL_PRIMER_APELLIDO,AFL_SEGUNDO_APELLIDO,AFL_PRIMER_NOMBRE,AFL_SEGUNDO_NOMBRE,...,GRP_FML_COTIZANTE_ID,PORTABILIDAD,COD_IPS_P,MTDLG_G_P,SUB_SISBEN_IV,MARCASISBENIV+MARCASISBENIII,CRUCE_BDEX_RNEC,MARCASISBENIV+MARCASISBENIII_2,Gr_Poblacional_Actual,N_Sisben_Actual
0,78742187,EPS025,NaN,NaN,CC,1118536120,CORREDOR,CACHAY,NELDA,TEREZA,...,-1,0,NaN,NaN,NaN,SIV(C10),1,SIV(C10),5,C10
1,73131072,EPS025,NaN,NaN,CC,74812263,CATAÑO,NaN,JOSE,AARON,...,-1,0,NaN,NaN,NaN,NaN,0,NaN,No sisben,
2,73127150,EPS025,NaN,NaN,CC,23901506,CACHAY,ROJAS,MARIA,ROSALBINA,...,-1,0,NaN,NaN,NaN,NaN,0,NaN,No sisben,
3,69245789,EPS025,NaN,NaN,CC,24111166,ACEVEDO,HERNANDEZ,TERESA,DE JESUS,...,-1,0,NaN,NaN,NaN,NaN,0,NaN,No sisben,
4,69388424,EPS025,NaN,NaN,CC,4184406,EGUE,CATAÑO,JOSE,PLUTARCO,...,-1,0,NaN,NaN,NaN,NaN,0,NaN,No sisben,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197165,102074082,EPSC25,NaN,NaN,CC,1116554624,TEJEDOR,JASPE,KAREN,LIZETH,...,-1,0,850100019001,2,B03,SIV(B03),0,SIV(B03),5,B03
197166,85208430,EPSC25,NaN,NaN,CC,74770436,ROZO,BERNAL,EULISIS,NaN,...,-1,0,851390042204,NaN,NaN,,0,SIII(18.06|1),No sisben,
197167,71880274,EPSC25,NaN,NaN,CC,1006446562,VIAZUS,AMARO,JESUS,ANTONIO,...,-1,0,NaN,2,B02,SIV(B02),0,SIV(B02),5,B02
197168,74483078,EPSC25,NaN,NaN,CC,1118532356,MALDONADO,PEREZ,JEYNNER,ALFONSO,...,-1,0,NaN,NaN,NaN,,0,SIII(60.81|0),No sisben,


In [11]:
# Guardar con nombres de hojas más descriptivos
output_file = os.path.join(Ruta_Salida, f"Datos_Procesados_{fecha_archivo}.xlsx")

print(f"💾 GUARDANDO DATAFRAMES PROCESADOS")
print(f"📁 Archivo: {output_file}")
print("="*60)

try:
    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
        # Guardar maestro_ADRES
        maestro_ADRES.to_excel(writer, sheet_name='Maestro_ADRES_Limpio', index=False)
        print(f"✅ Maestro ADRES guardado:")
        print(f"   📊 Filas: {len(maestro_ADRES):,}")
        print(f"   📊 Columnas: {len(maestro_ADRES.columns)}")
        
        # Guardar df_NS
        df_NS.to_excel(writer, sheet_name='NS_Validado', index=False)
        print(f"✅ DataFrame NS guardado:")
        print(f"   📊 Filas: {len(df_NS):,}")
        print(f"   📊 Columnas: {len(df_NS.columns)}")
        
        # Resumen de novedades en df_NS
        if 'NOVEDAD' in df_NS.columns:
            print(f"   📋 Tipos de novedad: {df_NS['NOVEDAD'].value_counts().to_dict()}")
    
    print(f"\n🎉 GUARDADO EXITOSO")
    print(f"📂 Ubicación: {output_file}")
    print(f"📋 Hojas creadas: 'Maestro_ADRES_Limpio', 'NS_Validado'")
    
except Exception as e:
    print(f"❌ Error al guardar: {e}")

💾 GUARDANDO DATAFRAMES PROCESADOS
📁 Archivo: C:\Users\crist\OneDrive - 891856000_CAPRESOCA E P S\Escritorio\Yesid Rincón Z\Traslados\Procesos BDUA\2025\10_Octubre\17\Datos_Procesados_17-10-2025.xlsx
✅ Maestro ADRES guardado:
   📊 Filas: 197,170
   📊 Columnas: 46
✅ DataFrame NS guardado:
   📊 Filas: 25,704
   📊 Columnas: 23
   📋 Tipos de novedad: {'N43': 24615, 'N46': 471, 'N21': 200, 'N39': 152, 'N03': 42, 'N09': 37, 'N02': 34, 'N14': 31, 'N04': 27, 'N25': 24, 'N01': 20, 'N32': 15, 'N19': 11, 'N41': 8, 'N12': 7, 'N36': 6, 'N31': 4}

🎉 GUARDADO EXITOSO
📂 Ubicación: C:\Users\crist\OneDrive - 891856000_CAPRESOCA E P S\Escritorio\Yesid Rincón Z\Traslados\Procesos BDUA\2025\10_Octubre\17\Datos_Procesados_17-10-2025.xlsx
📋 Hojas creadas: 'Maestro_ADRES_Limpio', 'NS_Validado'
